In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.preprocessing import MinMaxScaler

np.random.seed(42)

In [ ]:
df = pd.read_csv("AirPassengers.csv")

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df.shape

# EDA

In [ ]:
plt.figure(figsize=(25, 5))
plt.plot_date(df["Month"], df["#Passengers"], "b")
plt.xticks(rotation=90)
plt.show()

In [ ]:
df["Date"] = pd.to_datetime(df["Month"])
df.drop("Month", axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
df["Month"] = df["Date"].apply(lambda date: date.month)
df["Year"] = df["Date"].apply(lambda date: date.year)

In [ ]:
df.head()

In [ ]:
for year in df["Year"].unique():
    plt.figure()
    ax = sns.barplot(data=df[df["Year"] == year], x="Month", y="#Passengers")
    for container in ax.containers:
        ax.bar_label(container)
    plt.title(f"Passengers per Month in {year}")
    plt.show()

In [ ]:
passengers_per_years = df.groupby("Year")["#Passengers"].sum()

In [ ]:
passengers_per_years.plot(kind="bar")

In [ ]:
passengers_per_years.plot(kind="pie", autopct="%0.1f%%")

In [ ]:
passengers_per_months = df.groupby("Month")["#Passengers"].sum()

In [ ]:
passengers_per_months.plot(kind="bar")

In [ ]:
passengers_per_months.plot(kind="pie", autopct="%0.1f%%")

# Preprocess

In [ ]:
df.drop(["Year", "Month"], axis=1, inplace=True)

In [ ]:
df.set_index("Date", inplace=True)

In [ ]:
df.head()

In [ ]:
df.plot()

In [ ]:
passengers = df["#Passengers"]
passengers = np.array(passengers).reshape(-1, 1)

In [ ]:
scaler = MinMaxScaler()
passengers_scaled = scaler.fit_transform(passengers)

In [ ]:
train_size = int(len(passengers_scaled) * 0.7)
test_size = len(passengers_scaled) - train_size

In [ ]:
print("Train size:", train_size)
print("Test size:", test_size)

In [ ]:
train = passengers_scaled[0:train_size, :] 
test = passengers_scaled[train_size:len(passengers_scaled), :]

In [ ]:
def dataset(data, steps=1):
    data_x, data_y = [], []
    for i in range(len(data) - steps - 1):
        a = data[i:(i + steps), 0]
        b = data[i + steps, 0]
        data_x.append(a)
        data_y.append(b)
    data_x = np.array(data_x)
    data_y = np.array(data_y)
    return data_x, data_y

In [ ]:
X_train, y_train = dataset(train)
X_test, y_test = dataset(test)

In [ ]:
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Model

In [ ]:
model = Sequential()
model.add(LSTM(4, input_shape=(1,1)))
model.add(Dense(1))
model.compile(loss="mean_squared_error", optimizer="adam")

In [ ]:
history = model.fit(X_train, y_train, batch_size=1, epochs=25)

In [ ]:
model.save("LSTM_air.h5")

In [ ]:
y_pred = model.predict(X_test)
y_test = y_test.reshape(-1, 1)
y_test = scaler.inverse_transform(y_test)

In [ ]:
y_pred = scaler.inverse_transform(y_pred)

In [ ]:
plt.figure()
plt.plot(y_test, label="Actual")
plt.plot(y_pred, label="Predicted")
plt.legend()
plt.show()